In [1]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from datetime import date
import os.path
from selenium.webdriver.common.keys import Keys
import time
import mysql.connector
import pandas as pd
import numpy as np

sku = 'sku_blah'
quantity = 'quantity_blah'
price = 'price_blah'
location = 'Bangalore'
source = 'Reliance_fresh'

csvList = []
csvDict = {
    'sku': sku,
    'quantity': quantity,
    'price': price,
    'date': date,
    'location': location,
    'source': source,
}

today = date.today()
d1 = today.strftime( "%Y-%m-%d" )
date_1 = today.strftime( "%d_%m_%Y" )

if os.path.exists(r'Reliance.csv') == True:
    print('This file exists, not writing to the database')
else:
    print('This file does not exist and hence, writing to the database')
    my_url = [
        "https://www.jiomart.com/category/groceries/fruits-vegetables/fresh-vegetables",
        "https://www.jiomart.com/category/groceries/fruits-vegetables/fresh-fruits",
        "https://www.jiomart.com/category/groceries/fruits-vegetables/herbs-seasonings",
        "https://www.jiomart.com/category/groceries/fruits-vegetables/exotic-fruits-vegetables"
    ]

    browser = webdriver.Chrome( "/home/subhankar/Downloads/chromedriver" )

    for url in my_url:
        browser.get( url )
        landing_page = browser.find_element_by_tag_name( "html" )
        time.sleep( 4 )
        browser.maximize_window()

        #your_city_elem = browser.find_element_by_id( "localitySelectionDrop" )
        #your_city_elem.send_keys( "Bangalore" )
        #your_city_elem.send_keys( Keys.ENTER )

        try :
            search_pin = browser.find_element_by_id("rel_pincode")
            search_pin.send_keys( "560003" )
            time.sleep( 1 )
            search_pin.send_keys( Keys.ENTER )
        except:
            search_pin = browser.find_element_by_id("delivering_to")
            search_pin.send_keys( "560003" )
            time.sleep( 1 )
            search_pin.send_keys( Keys.ENTER )

        time.sleep( 1 )
        browser.get( url )
        scroll_down = browser.find_element_by_tag_name( "html" )
        time.sleep( 1 )
        scroll_down.send_keys( Keys.END )
        time.sleep( 1 )
        scroll_down.send_keys( Keys.END )
        time.sleep( 1 )
        scroll_down.send_keys( Keys.END )


        res = browser.execute_script( "return document.documentElement.outerHTML" )
        page_soup = soup( res, "html.parser" )


        containers = page_soup.findAll( "div", {"class": "cat-item"} )
        for container in containers:
            header_container = container.findAll( "span", {"class": "clsgetname"} )
            head = header_container[0].text.split("-")
            header = head[0]
            quantity = ''
            try:
                price_container = container.findAll("span", {"id":"final_price"})
                price = price_container[0].text
            except:
                price = 0

            csvDict['sku'] = header
            csvDict['quantity'] = quantity
            csvDict['price'] = price
            csvDict['date'] = d1
            csvDict['location'] = location
            csvDict['source'] = source

            csvList.append( csvDict.copy() )

    print( "the no of items is", len( csvList ) )
    print( csvList )
    browser.close()
    reliance = open( "reliance_BLR_" + date_1 + ".csv", "w" )

    def write_to_csv(output):
        with open( "reliance_BLR_" + date_1 + ".csv", "w" ) as f:
            f.write( "Name,Price(Rs),Quantity,Website,date,location\n" )
            for line in output:
                f.write( f"{line['sku']},{line['price']},{line['quantity']},{line['source']},{line['date']},{line['location']}\n" )


    reliance.close()
    write_to_csv( csvList )
    df=pd.read_csv("reliance_BLR_" + date_1 + ".csv")
    df['Quantity']=df['Quantity'].replace( np.nan,'', regex=True)
    df['New_quantity']=df[df['Quantity'] == ''].Name.str.split(r"([0-9])",n=1,expand=True)[1]+df[df['Quantity'] == ''].Name.str.split(r"([0-9])",n=1,expand=True)[2]
    df['Quantity']=df['Quantity'].replace(r'', np.nan, regex=True)
    df.Quantity.fillna(df.New_quantity, inplace=True)
    df.Quantity.fillna("1 units", inplace=True)
    df=df.drop(['New_quantity'], axis=1)
    df['Name'].update(df.Name.str.split(r"([0-9])",n=1,expand=True)[0])
    df.loc[df['Name'].str.contains('kg', case=False), 'Quantity'] = '1 kg' 
    #df.to_csv('namdhari_bangalore5_unprocessed.csv', index=False)
    df.to_csv("reliance_BLR_" + date_1 + ".csv",index=False)
    df.to_csv(date_1+'.csv',mode='a', header=False,index=False)

This file does not exist and hence, writing to the database
the no of items is 107
[{'sku': 'Root Vegetables', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Vegetables Others', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Beans', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Brinjal', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Cabbage', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Capsicum', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Carrot', 'quantity': '', 'price': 0, 'date': '2020-06-26', 'location': 'Bangalore', 'source': 'Reliance_fresh'}, {'sku': 'Cauliflower', 'quantity': '', 

In [25]:

df.to_csv("reliance_BLR_" + date_1 + ".csv",index=False)
df

,Name,Price(Rs),Quantity,Website,date,location
0,Root Vegetables,0,1 units,Reliance_fresh,2020-06-25,Bangalore
1,Vegetables Others,0,1 units,Reliance_fresh,2020-06-25,Bangalore
2,Brinjal,0,1 units,Reliance_fresh,2020-06-25,Bangalore
3,Cabbage,0,1 units,Reliance_fresh,2020-06-25,Bangalore
4,Capsicum,0,1 units,Reliance_fresh,2020-06-25,Bangalore
5,Carrot,0,1 units,Reliance_fresh,2020-06-25,Bangalore
6,Cauliflower,0,1 units,Reliance_fresh,2020-06-25,Bangalore
7,Corn,0,1 units,Reliance_fresh,2020-06-25,Bangalore
8,Cucumber,0,1 units,Reliance_fresh,2020-06-25,Bangalore
9,Gourds,0,1 units,Reliance_fresh,2020-06-25,Bangalore
